<a href="https://colab.research.google.com/github/Ivoryle82/Disaster-Relief/blob/main/Disaster_Relief_Ivory_Le.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Disaster Relief Project
The data set for this project consists of tweets from people experiencing natural disasters around the world. The tweets are classified by category some are further classified by having some sort of need.


In [ ]:
#@title Starter libraries (double click to take a look) { display-mode: "form" }
# useful for opening files
import gdown
import zipfile

import os # accessing parts of your operating system
import re
import sys

# data visualization + manipulation -- we've seen these many times
import numpy as np
import pandas as pd

# plotting libraries
import seaborn as sns
import matplotlib.pyplot as plt

# regression models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.preprocessing import PolynomialFeatures # for polynomial model
from sklearn.pipeline import Pipeline

# classification models
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

# more sklearn model making
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

from collections import Counter

# NLP
import string
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords' ,quiet=True)
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, TweetTokenizer
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
import wordcloud
import tweepy
!python -m spacy download en_core_web_md
import en_core_web_md
text_to_nlp = spacy.load('en_core_web_md')

# NN models
import tensorflow as tf
import tensorflow_datasets as tfds

import keras
from tensorflow.keras import layers , activations , models , preprocessing, utils

# sequence data
from keras import Input, Model
from keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import RMSprop
from keras.utils import pad_sequences

# text
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer

# NN and CNN
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout
from keras.layers import Activation, MaxPooling2D, Dropout, Flatten, Reshape
from keras.wrappers.scikit_learn import KerasClassifier
import keras.optimizers as optimizers
from keras.activations import softmax
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.applications import VGG16, VGG19, ResNet50, DenseNet121

import warnings
warnings.filterwarnings('ignore')


2023-08-15 18:49:01.635514: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
#@title Load your dataset { display-mode: "form" }
import pandas as pd

!wget -q --show-progress 'https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Disaster%20Relief/disaster_data.csv'
disaster_tweets = pd.read_csv('disaster_data.csv',encoding ="ISO-8859-1")

disaster_data.csv.3 100%[===================>] 231.23K  --.-KB/s    in 0.001s  


In [ ]:
# This function prints out a table containing all the tweets, along with their category labels
disaster_tweets

,tweet_id,text,category,need_or_resource
0,ca9e24c8-396d-4502-8b45-18895df5333e_0,"Donations of batteries, flashlights, and clean...",Energy,need
1,twitter_resource_tweets_1692,I want hurricane Sandy to cone so I can be stu...,None,NaN
2,625b46e2-0b81-41ea-826e-4535fe9b39b8,"Hi, I can help prepare food, serve food, offer...",Food,resource
3,twitter_resource_tweets_1699,I cant believe Sandy.....,None,NaN
4,c3bfea72-d377-445c-b4b8-e8ebca0e7fbb,I have children and adult clothes including ja...,Water,resource
...,...,...,...,...
1396,4954cd07-1796-4841-a79a-9edf7faf9521,We were here for the marathon and would like t...,Food,resource
1397,425e0f7e-eebc-4bc7-9027-34b3298ee084,I am writing on behalf of friends stranded in ...,Medical,need
1398,twitter_need_tweets_4630,Finally at my Uncle's house in Brooklyn. Gonna...,None,NaN
1399,twitter_resource_tweets_2745,Two years in a raw... #Irene then #Sandy... Co...,None,NaN


In [ ]:
#this fuctions print out only the texts with their type
needed_columns = ['text','need_or_resource','category']
disaster_tweets = disaster_tweets[needed_columns]
disaster_tweets.head()

,text,need_or_resource,category
0,"Donations of batteries, flashlights, and clean...",need,Energy
1,I want hurricane Sandy to cone so I can be stu...,NaN,None
2,"Hi, I can help prepare food, serve food, offer...",resource,Food
3,I cant believe Sandy.....,NaN,None
4,I have children and adult clothes including ja...,resource,Water


# The texts that are usually being used for each type



In [ ]:
type =  "need"#@param {type:"string"}
for t in disaster_tweets[disaster_tweets['need_or_resource'] == type]['text'].head(20).values:
    print (t)

Donations of batteries, flashlights, and cleaning supplies are always welcome at our donation drop off at 23-74 38th st btwn 23rd ave & Astoria Blvd.  Thanks!
spooky: the total lack of power from 34th st - south. #sandy
Will these storms ever give L.I. a break? First sandy, now a nor'easter. Still no power in long beach expected for a few wks. #c' monman
We're very concerned about our 90-year old grandmother *************** who lives by herself in her apartment at *********************************************** without power or running water. She said her neighbors are helping her, but it's hard to know if she's really ok from the phone and we're concerned her cell will run out of power. Her number is ****************.
Enjoying life in the dark.... (@ Hurricane Sandy's Power Outage) http://t.co/ZdCwWy8Q
************************. (Needs confirmation of location.) May need food.
My cousin and his wife are stuck. There is no electricity (and thus no elevator)or phone (or cell phone servic

# Preparing our data

In [ ]:
def is_needed_tweet(need_or_resource):
    if need_or_resource == 'need':
        return True
    else:
        return False

disaster_tweets['is_needed_tweet'] = disaster_tweets['need_or_resource'].apply(is_needed_tweet)
disaster_tweets.head()

,text,need_or_resource,category,is_needed_tweet
0,"Donations of batteries, flashlights, and clean...",need,Energy,True
1,I want hurricane Sandy to cone so I can be stu...,NaN,None,False
2,"Hi, I can help prepare food, serve food, offer...",resource,Food,False
3,I cant believe Sandy.....,NaN,None,False
4,I have children and adult clothes including ja...,resource,Water,False


In [ ]:
X_text = disaster_tweets['text']
y = disaster_tweets['is_needed_tweet']

In [ ]:
def tokenize(text):
    clean_tokens = []
    for token in text_to_nlp(text):
        if (not token.is_stop) & (token.lemma_ != '-PRON-') & (not token.is_punct):
            clean_tokens.append(token.lemma_)
    return clean_tokens


In [ ]:
bow_transformer = CountVectorizer(analyzer=tokenize, max_features=1400).fit(X_text)

In [ ]:
bow_transformer.vocabulary_

{'donation': 570,
 'battery': 359,
 'flashlight': 638,
 'cleaning': 465,
 'supply': 1225,
 'welcome': 1357,
 'drop': 579,
 '23': 32,
 '74': 57,
 '38th': 42,
 'st': 1194,
 'btwn': 406,
 '23rd': 33,
 'ave': 342,
 'Astoria': 86,
 'Blvd': 97,
 ' ': 0,
 'thank': 1251,
 'want': 1344,
 'hurricane': 762,
 'Sandy': 235,
 'cone': 493,
 'stick': 1206,
 'house': 755,
 'family': 622,
 'hi': 739,
 'help': 736,
 'prepare': 1031,
 'food': 648,
 'serve': 1133,
 'offer': 951,
 'clean': 464,
 'assistance': 337,
 'tool': 1271,
 'hand': 716,
 'bicycle': 369,
 'maybe': 878,
 'delivery': 544,
 'walk': 1342,
 'distance': 564,
 'Red': 225,
 'Hook': 171,
 'not': 941,
 'child': 452,
 'adult': 303,
 'clothe': 472,
 'include': 776,
 'jacket': 795,
 'blanket': 374,
 'shoe': 1145,
 'bottle': 388,
 'water': 1349,
 'small': 1166,
 'like': 833,
 'donate': 569,
 'need': 925,
 'toy': 1277,
 'kid': 805,
 'know': 811,
 'red': 1072,
 'cross': 523,
 'directly': 559,
 'advance': 304,
 'scary': 1118,
 'hurricanesandy': 763,
 '

In [ ]:
len(bow_transformer.vocabulary_)

1400

In [ ]:
X = bow_transformer.transform(X_text)

In [ ]:
pd.DataFrame(X.toarray())

,0,1,2,3,4,5,6,7,8,9,...,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1399,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


# Creating a Baseline Classifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

LogisticRegression()

In [ ]:
y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print (accuracy)

0.8825622775800712


# Final Result

In [ ]:
for i in range(len(y_test)):
    print("Tweet:", X_text.iloc[i])  # Print the actual tweet
    prediction = logistic_model.predict(bow_transformer.transform([X_text.iloc[i]]))
    if prediction:
        print("This was a need tweet")
    else:
        print("This was a resource tweet!")
    print("\n---\n")  # Separate output for better readability


Tweet: Donations of batteries, flashlights, and cleaning supplies are always welcome at our donation drop off at 23-74 38th st btwn 23rd ave & Astoria Blvd.  Thanks!
This was a need tweet

---

Tweet: I want hurricane Sandy to cone so I can be stuck in my house with my family :) ..............NOT !!!!!
This was a resource tweet!

---

Tweet: Hi, I can help prepare food, serve food, offer clean up assistance but don't necessarily have any tools, just my hands and willingness to help. I only have a bicycle and could maybe help with deliveries. I'm within walking distance of Red Hook.
This was a resource tweet!

---

Tweet: I cant believe Sandy.....
This was a resource tweet!

---

Tweet: I have children and adult clothes including jacket , blanket, shoes , and bottle water
This was a resource tweet!

---

Tweet: I have two small children but we would like to donate to families who need clothes or food or even toys for kids! But we don't know where and would rather not donate to red cross

In [ ]:
X = disaster_tweets['text']
y = disaster_tweets['category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1400)

# Transform text data into numerical features
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train a Logistic Regression classifier
logistic_model = LogisticRegression(max_iter=1400)
logistic_model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = logistic_model.predict(X_test_tfidf)

# Evaluate the model
print(classification_report(y_test, y_pred))







              precision    recall  f1-score   support

      Energy       0.75      0.44      0.56        34
        Food       0.84      0.96      0.90       130
     Medical       1.00      0.22      0.36         9
        None       0.79      0.93      0.85        85
       Water       1.00      0.48      0.65        23

    accuracy                           0.83       281
   macro avg       0.88      0.61      0.66       281
weighted avg       0.83      0.83      0.81       281

